<a href="https://colab.research.google.com/github/omnimanwani/DesignProject/blob/main/version1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install jsonlines

In [ ]:
# import json
# import pandas as pd
# import logging
# from tqdm import tqdm
# import re
# from typing import Dict, List, Tuple, Optional
# from dataclasses import dataclass
# import torch
# from transformers import pipeline

# @dataclass
# class ProcessingResult:
#     entities: List[Dict[str, str]]
#     intent: str
#     confidence: float
#     analysis_details: Dict

# class PromptMovieProcessor:
#     """Process movie dialogue data using prompts and LLMs."""

#     def __init__(self):
#         """Initialize the processor with models and prompts."""
#         self._initialize_logging()
#         self._setup_models()
#         self._initialize_prompts()

#     def _initialize_logging(self) -> None:
#         """Set up logging configuration."""
#         self.logger = logging.getLogger(__name__)
#         self.logger.setLevel(logging.INFO)

#         # Create console handler with formatting
#         console_handler = logging.StreamHandler()
#         console_handler.setLevel(logging.INFO)
#         formatter = logging.Formatter(
#             '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
#         )
#         console_handler.setFormatter(formatter)
#         self.logger.addHandler(console_handler)

#         # Also log to file
#         file_handler = logging.FileHandler('movie_processor.log')
#         file_handler.setLevel(logging.INFO)
#         file_handler.setFormatter(formatter)
#         self.logger.addHandler(file_handler)

#     def _initialize_prompts(self) -> None:
#         """Initialize system and task-specific prompts."""
#         self.entity_prompt_template = """Analyze this movie-related message and identify entities. Focus on:

# PERSON: Actors, directors, characters
# WORK_OF_ART: Movies, TV shows, series titles
# ORG: Studios, production companies
# GENRE: Movie genres
# ATTRIBUTE: Movie qualities or characteristics

# Message: "{text}"

# List all entities found in this format:
# Entity: [exact text from message]
# Type: [entity type]
# Explanation: [why this is that type of entity]

# Only include entities you're confident about."""

#         self.intent_prompt_template = """Analyze this movie-related message and identify the main conversational intent. Consider these categories:

# 1. RECOMMENDATION_REQUEST: Asking for movie suggestions
# Example: "Can you suggest movies like Inception?"

# 2. PREFERENCE_QUESTION: Asking about movie preferences
# Example: "What kind of movies do you enjoy?"

# 3. OPINION_SHARING: Expressing thoughts about movies
# Example: "I loved The Dark Knight, it was amazing!"

# 4. INFORMATION_REQUEST: Asking for movie information
# Example: "Who directed Pulp Fiction?"

# 5. CRITIQUE: Detailed movie criticism
# Example: "The pacing was slow and the plot had holes..."

# 6. RECOMMENDATION_GIVING: Suggesting movies
# Example: "You should watch Shawshank Redemption"

# 7. FACTUAL_STATEMENT: Stating movie facts
# Example: "Titanic won 11 Academy Awards"

# 8. GENERAL_DISCUSSION: General movie chat
# Example: "Movies these days are so different"

# Message: "{text}"

# Identify:
# 1. Primary Intent: [category]
# 2. Confidence (0-1): [score]
# 3. Reasoning: [brief explanation]"""

#         self.movie_context_prompt = """Consider this message in the context of a movie recommendation conversation:

# Previous context about movies: {context}
# Current message: "{text}"

# Analyze:
# 1. Are there implicit movie references?
# 2. Any indirect mentions of movie elements?
# 3. What aspects of movies are being discussed?

# Format findings as:
# Referenced Movies: [list]
# Implied Genres: [list]
# Discussion Topics: [list]"""

#     def _setup_models(self) -> None:
#         """Set up necessary models from Hugging Face."""
#         self.logger.info("Setting up models...")

#         try:
#             # Use BERT for zero-shot classification
#             self.classifier = pipeline(
#                 "zero-shot-classification",
#                 model="facebook/bart-large-mnli",
#                 device="cuda" if torch.cuda.is_available() else "cpu"
#             )

#             # Use RoBERTa for NER
#             self.ner_pipeline = pipeline(
#                 "ner",
#                 model="jean-baptiste/roberta-large-ner-english",
#                 device="cuda" if torch.cuda.is_available() else "cpu"
#             )

#             self.logger.info("Models loaded successfully")
#         except Exception as e:
#             self.logger.error(f"Error loading models: {str(e)}")
#             raise

#     def process_batch(self, texts: List[str], batch_size: int = 32) -> List[ProcessingResult]:
#         """Process a batch of texts with progress bar."""
#         results = []
#         self.logger.info(f"Processing batch of {len(texts)} texts")

#         try:
#             for i in tqdm(range(0, len(texts), batch_size)):
#                 batch = texts[i:i + batch_size]
#                 batch_results = [
#                     self.extract_entities_and_intents(text)
#                     for text in batch
#                 ]
#                 results.extend(batch_results)

#             self.logger.info(f"Successfully processed {len(results)} texts")
#             return results

#         except Exception as e:
#             self.logger.error(f"Error in batch processing: {str(e)}")
#             raise

#     def save_results(self, results: List[ProcessingResult], output_file: str) -> None:
#         """Save processing results to JSON file."""
#         try:
#             output_data = [
#                 {
#                     "entities": result.entities,
#                     "intent": result.intent,
#                     "confidence": result.confidence,
#                     "analysis_details": result.analysis_details
#                 }
#                 for result in results
#             ]

#             with open(output_file, 'w') as f:
#                 json.dump(output_data, f, indent=2)

#             self.logger.info(f"Results saved to {output_file}")

#         except Exception as e:
#             self.logger.error(f"Error saving results: {str(e)}")
#             raise

#     def load_data(self, input_file: str) -> List[str]:
#         """Load input data from file (CSV or JSON)."""
#         try:
#             if input_file.endswith('.csv'):
#                 df = pd.read_csv(input_file)
#                 texts = df['text'].tolist()  # Assumes 'text' column
#             elif input_file.endswith('.json'):
#                 with open(input_file, 'r') as f:
#                     data = json.load(f)
#                 texts = [item['text'] for item in data]  # Assumes list of dicts with 'text' key
#             else:
#                 raise ValueError("Unsupported file format. Use CSV or JSON.")

#             self.logger.info(f"Loaded {len(texts)} texts from {input_file}")
#             return texts

#         except Exception as e:
#             self.logger.error(f"Error loading data: {str(e)}")
#             raise

#     def extract_entities_and_intents(self,
#                                    text: str,
#                                    context: Optional[List[str]] = None) -> ProcessingResult:
#         """Extract entities and classify intent using prompts and models."""
#         try:
#             # 1. Entity Recognition with Prompt
#             entity_prompt = self.entity_prompt_template.format(text=text)
#             entity_results = self._process_with_prompt_ner(text, entity_prompt)

#             # 2. Intent Classification with Prompt
#             intent_prompt = self.intent_prompt_template.format(text=text)
#             intent_result = self._process_with_prompt_intent(text, intent_prompt)

#             # 3. Additional Context Analysis (if context provided)
#             context_analysis = {}
#             if context:
#                 context_prompt = self.movie_context_prompt.format(
#                     context="\n".join(context[-3:]),  # Use last 3 messages
#                     text=text
#                 )
#                 context_analysis = self._analyze_movie_context(text, context_prompt)

#             # Combine results
#             result = ProcessingResult(
#                 entities=entity_results['entities'],
#                 intent=intent_result['intent'],
#                 confidence=intent_result['confidence'],
#                 analysis_details={
#                     'entity_explanations': entity_results['explanations'],
#                     'intent_reasoning': intent_result['reasoning'],
#                     'context_analysis': context_analysis
#                 }
#             )

#             return result

#         except Exception as e:
#             self.logger.error(f"Error in processing: {str(e)}")
#             return ProcessingResult(
#                 entities=[],
#                 intent="unknown",
#                 confidence=0.0,
#                 analysis_details={}
#             )

#     def _process_with_prompt_ner(self, text: str, prompt: str) -> Dict:
#         """Process text for NER using prompt-based approach."""
#         # Zero-shot NER with custom labels
#         ner_labels = [
#             "PERSON - actor/director/character",
#             "WORK_OF_ART - movie/show title",
#             "ORG - movie studio/company",
#             "GENRE - movie genre",
#             "ATTRIBUTE - movie characteristic"
#         ]

#         results = self.classifier(
#             text,
#             candidate_labels=ner_labels,
#             multi_label=True
#         )

#         # Process results into entities
#         entities = []
#         explanations = []

#         for label, score in zip(results['labels'], results['scores']):
#             if score > 0.7:  # Confidence threshold
#                 entity_type = label.split(' - ')[0]
#                 # Extract the actual entity text using regex patterns
#                 if entity_type == "PERSON":
#                     matches = re.finditer(r'[A-Z][a-z]+(?:\s+[A-Z][a-z]+)*', text)
#                 elif entity_type == "WORK_OF_ART":
#                     matches = re.finditer(r'(?:The\s)?[A-Z][^.!?]*?(?:movie|film|show|series)?', text)
#                 else:
#                     matches = re.finditer(r'\b[A-Z][a-z]+(?:\s+[A-Z][a-z]+)*\b', text)

#                 for match in matches:
#                     entities.append({
#                         "text": match.group(),
#                         "type": entity_type,
#                         "confidence": score
#                     })
#                     explanations.append(f"{match.group()} identified as {entity_type} with confidence {score:.2f}")

#         return {
#             "entities": entities,
#             "explanations": explanations
#         }

#     def _process_with_prompt_intent(self, text: str, prompt: str) -> Dict:
#         """Process text for intent classification using prompt-based approach."""
#         intent_labels = [
#             "RECOMMENDATION_REQUEST",
#             "PREFERENCE_QUESTION",
#             "OPINION_SHARING",
#             "INFORMATION_REQUEST",
#             "CRITIQUE",
#             "RECOMMENDATION_GIVING",
#             "FACTUAL_STATEMENT",
#             "GENERAL_DISCUSSION"
#         ]

#         result = self.classifier(
#             text,
#             candidate_labels=intent_labels
#         )

#         return {
#             "intent": result['labels'][0],
#             "confidence": result['scores'][0],
#             "reasoning": f"Classified as {result['labels'][0]} based on message content and pattern matching"
#         }

#     def _analyze_movie_context(self, text: str, prompt: str) -> Dict:
#         """Analyze movie-specific context and references."""
#         # Use zero-shot classification for movie-related aspects
#         aspect_labels = [
#             "discusses_plot",
#             "mentions_actors",
#             "compares_movies",
#             "asks_for_recommendations",
#             "shares_opinion"
#         ]

#         result = self.classifier(
#             text,
#             candidate_labels=aspect_labels,
#             multi_label=True
#         )

#         return {
#             "aspects": [label for label, score in zip(result['labels'], result['scores']) if score > 0.5],
#             "confidence_scores": dict(zip(result['labels'], result['scores']))
#         }

# def main():
#     """Example usage with prompts."""
#     processor = PromptMovieProcessor()

#     # Example usage with a single text
#     text = "I loved Inception! Can you recommend similar mind-bending movies like what Christopher Nolan directs?"
#     result = processor.extract_entities_and_intents(text)
#     print("\nSingle text analysis:")
#     print("Entities found:", json.dumps(result.entities, indent=2))
#     print("Intent:", result.intent)
#     print("Confidence:", result.confidence)
#     print("Analysis details:", json.dumps(result.analysis_details, indent=2))

#     # Example batch processing
#     texts = [
#         "The Dark Knight is probably Nolan's best film.",
#         "Can you suggest some good sci-fi movies?",
#         "I think Marvel movies are getting repetitive.",
#     ]
#     print("\nBatch processing example:")
#     results = processor.process_batch(texts, batch_size=2)

#     # Save results
#     processor.save_results(results, "movie_analysis_results.json")

# if __name__ == "__main__":
#     main()

2024-11-07 05:09:00,042 - __main__ - INFO - Setting up models...
2024-11-07 05:09:00,042 - __main__ - INFO - Setting up models...
2024-11-07 05:09:00,042 - __main__ - INFO - Setting up models...
INFO:__main__:Setting up models...
2024-11-07 05:09:03,899 - __main__ - INFO - Models loaded successfully
2024-11-07 05:09:03,899 - __main__ - INFO - Models loaded successfully
2024-11-07 05:09:03,899 - __main__ - INFO - Models loaded successfully
INFO:__main__:Models loaded successfully
2024-11-07 05:09:16,011 - __main__ - INFO - Processing batch of 3 texts
2024-11-07 05:09:16,011 - __main__ - INFO - Processing batch of 3 texts
2024-11-07 05:09:16,011 - __main__ - INFO - Processing batch of 3 texts
INFO:__main__:Processing batch of 3 texts



Single text analysis:
Entities found: [
  {
    "text": "Inception",
    "type": "ATTRIBUTE",
    "confidence": 0.9421381950378418
  },
  {
    "text": "Can",
    "type": "ATTRIBUTE",
    "confidence": 0.9421381950378418
  },
  {
    "text": "Christopher Nolan",
    "type": "ATTRIBUTE",
    "confidence": 0.9421381950378418
  }
]
Intent: PREFERENCE_QUESTION
Confidence: 0.23613303899765015
Analysis details: {
  "entity_explanations": [
    "Inception identified as ATTRIBUTE with confidence 0.94",
    "Can identified as ATTRIBUTE with confidence 0.94",
    "Christopher Nolan identified as ATTRIBUTE with confidence 0.94"
  ],
  "intent_reasoning": "Classified as PREFERENCE_QUESTION based on message content and pattern matching",
  "context_analysis": {}
}

Batch processing example:


100%|██████████| 2/2 [00:24<00:00, 12.16s/it]
2024-11-07 05:09:40,358 - __main__ - INFO - Successfully processed 3 texts
2024-11-07 05:09:40,358 - __main__ - INFO - Successfully processed 3 texts
2024-11-07 05:09:40,358 - __main__ - INFO - Successfully processed 3 texts
INFO:__main__:Successfully processed 3 texts
2024-11-07 05:09:40,364 - __main__ - INFO - Results saved to movie_analysis_results.json
2024-11-07 05:09:40,364 - __main__ - INFO - Results saved to movie_analysis_results.json
2024-11-07 05:09:40,364 - __main__ - INFO - Results saved to movie_analysis_results.json
INFO:__main__:Results saved to movie_analysis_results.json


In [ ]:
import json
import pandas as pd
import logging
from tqdm import tqdm
from typing import Dict, List, Optional
from dataclasses import dataclass
import torch
from transformers import pipeline
import jsonlines
import re

@dataclass
class MessageAnalysisResult:
    message_id: int
    conversation_id: str
    text: str
    time_offset: int
    intent: str
    confidence: float
    entities: List[Dict[str, str]]
    referenced_movies: List[str]
    analysis_details: Dict

class MovieDialogAnalyzer:
    """Analyze movie dialogue messages for intents and entities."""

    def __init__(self):
        self._initialize_logging()
        self._setup_models()
        self._initialize_prompts()

    def _initialize_logging(self) -> None:
        self.logger = logging.getLogger(__name__)
        self.logger.setLevel(logging.INFO)

        console_handler = logging.StreamHandler()
        console_handler.setLevel(logging.INFO)
        formatter = logging.Formatter(
            '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
        )
        console_handler.setFormatter(formatter)
        self.logger.addHandler(console_handler)

        file_handler = logging.FileHandler('movie_dialog_analyzer.log')
        file_handler.setLevel(logging.INFO)
        file_handler.setFormatter(formatter)
        self.logger.addHandler(file_handler)

    def _initialize_prompts(self) -> None:
        self.entity_prompt_template = """Analyze this movie-related message and identify entities. Focus on:

PERSON: Actors, directors, characters
WORK_OF_ART: Movies, TV shows, series titles
ORG: Studios, production companies
GENRE: Movie genres
ATTRIBUTE: Movie qualities or characteristics

Message: "{text}"

List all entities found in this format:
Entity: [exact text from message]
Type: [entity type]
Confidence: [confidence score]
Explanation: [why this is that type of entity]

Only include entities you're confident about."""

        self.intent_prompt_template = """Analyze this movie-related message and identify the main conversational intent. Consider these categories:

Intent Name: Initiate
Intent Description: Start a dialogue with the system.
Intent Example: ["hello", "hi", "hey", "greetings", "start", "begin"]

Intent Name: Chit-chat
Intent Description: Utterances unrelated to the recommendation goal.
Intent Example: ["how are you", "what's up", "nice weather", "that's interesting", "really","weather", "nice", "thanks", "thank you"]

Intent Name: Provide Preferences
Intent Description: Share preferences with the system.
Intent Example: ["like", "enjoy", "prefer", "favorite", "love", "hate", "dislike"]

Intent Name: Revise Preferences
Intent Description: Revise previously stated preferences.
Intent Example: ["actually", "instead", "rather", "change", "different", "not that"]

Intent Name: Ask for Recommendation
Intent Description: Obtain system suggestions.
Intent Example: ["recommend", "suggest", "what should", "can you find", "looking for"]

Intent Name: Obtain Explanation
Intent Description: Learn more about why something was recommended.
Intent Example: ["why", "how come", "explain", "reason", "because"]

Intent Name: Obtain Details
Intent Description: Ask about more details of a recommended object.
Intent Example: ["what is", "tell me about", "details", "more info", "description"]

Intent Name: Feedback on Recommendation
Intent Description: Give feedback on the provided recommendation(s).
Intent Example: ["good suggestion", "bad suggestion", "not what", "perfect", "exactly"]

Intent Name: Restart
Intent Description: Restart the dialogue.
Intent Example: ["start over", "begin again", "restart", "fresh", "new search"]

Intent Name: Accept Recommendation
Intent Description: Accept one of the recommendations.
Intent Example: ["that works", "sounds good", "perfect", "i'll take", "great choice"]

Intent Name: Quit
Intent Description: Terminate the conversation.
Intent Example: ["goodbye", "bye", "that's all", "done", "exit", "quit"]

Message: "{text}"

Identify:
1. Primary Intent: [category]
2. Confidence (0-1): [score]
3. Reasoning: [brief explanation]"""

    def _setup_models(self) -> None:
        """Set up necessary models from Hugging Face."""
        self.logger.info("Setting up models...")
        try:
            self.classifier = pipeline(
                "zero-shot-classification",
                model="facebook/bart-large-mnli",
                device="cuda" if torch.cuda.is_available() else "cpu"
            )

            self.ner_pipeline = pipeline(
                "ner",
                model="jean-baptiste/roberta-large-ner-english",
                device="cuda" if torch.cuda.is_available() else "cpu"
            )
            self.logger.info("Models loaded successfully")
        except Exception as e:
            self.logger.error(f"Error loading models: {str(e)}")
            raise

    def _process_with_prompt_intent(self, text: str, prompt: str) -> Dict:
        """Process text for intent classification using prompt-based approach."""
        intent_labels = [
            "Initiate",
            "Chit-chat",
            "Provide Preferences",
            "Revise Preferences",
            "Ask for Recommendation",
            "Obtain Explanation",
            "Obtain Details",
            "Feedback on Recommendation",
            "Restart",
            "Accept Recommendation",
            "Quit"
        ]

        result = self.classifier(
            text,
            candidate_labels=intent_labels
        )

        return {
            "intent": result['labels'][0],
            "confidence": result['scores'][0]
        }

    def _process_dialogue(self, dialogue: Dict) -> List[MessageAnalysisResult]:
        """Process a single dialogue and analyze all its messages."""
        results = []
        movie_mentions = dialogue.get('movieMentions', {})
        initiator_id = dialogue.get('initiatorWorkerId')

        if not initiator_id:
            self.logger.warning(f"No initiator ID found for dialogue {dialogue.get('conversationId', 'unknown')}")
            return results

        # Filter only initiator's messages
        initiator_messages = [
            message for message in dialogue.get('messages', [])
            if message.get('senderWorkerId') == initiator_id
        ]

        for message in initiator_messages:
            processed_text = self._replace_movie_tokens(message.get('text', ''), movie_mentions)

            if len(processed_text.strip()) < 2:
                continue

            referenced_movies = self._extract_referenced_movies(message.get('text', ''), movie_mentions)

            result = self._analyze_message(
                message_id=message.get('messageId', 0),
                conversation_id=dialogue.get('conversationId', ''),
                text=processed_text,
                original_text=message.get('text', ''),
                time_offset=message.get('timeOffset', 0),
                referenced_movies=referenced_movies
            )

            results.append(result)

        return results

    def _replace_movie_tokens(self, text: str, movie_mentions: Dict[str, str]) -> str:
        """Replace @movieID tokens with actual movie names."""
        processed_text = text
        for movie_id, movie_name in movie_mentions.items():
            processed_text = processed_text.replace(f"@{movie_id}", movie_name)
        return processed_text

    def _extract_referenced_movies(self, text: str, movie_mentions: Dict[str, str]) -> List[str]:
        """Extract list of movies referenced in the message."""
        referenced_movies = []
        for movie_id, movie_name in movie_mentions.items():
            if f"@{movie_id}" in text:
                referenced_movies.append(movie_name)
        return referenced_movies

    def _process_with_prompt_ner(self, text: str, prompt: str) -> Dict:
        """Process text for NER using prompt-based approach with confidence scores."""
        ner_labels = [
            "PERSON - actor/director/character",
            "WORK_OF_ART - movie/show title",
            "ORG - movie studio/company",
            "GENRE - movie genre",
            "ATTRIBUTE - movie characteristic"
        ]

        results = self.classifier(
            text,
            candidate_labels=ner_labels,
            multi_label=True
        )

        entities = []

        for label, score in zip(results['labels'], results['scores']):
            if score > 0.5:
                entity_type = label.split(' - ')[0]

                if entity_type == "PERSON":
                    pattern = r'[A-Z][a-z]+(?:\s+[A-Z][a-z]+)*'
                elif entity_type == "WORK_OF_ART":
                    pattern = r'(?:The\s)?[A-Z][^.!?]*?(?:movie|film|show|series)?'
                else:
                    pattern = r'\b[A-Z][a-z]+(?:\s+[A-Z][a-z]+)*'

                matches = re.finditer(pattern, text)

                for match in matches:
                    entity_text = match.group()
                    entity_confidence = self._calculate_entity_confidence(
                        text=text,
                        entity_text=entity_text,
                        entity_type=entity_type,
                        base_score=score
                    )

                    if entity_confidence > 0.6:  # Threshold for final entities
                        entities.append({
                            "entity": entity_text,
                            "type": entity_type,
                            "confidence": round(entity_confidence, 3)
                        })

        unique_entities = {}
        for entity in entities:
            key = (entity['entity'], entity['type'])
            if key not in unique_entities or entity['confidence'] > unique_entities[key]['confidence']:
                unique_entities[key] = entity

        return {"entities": list(unique_entities.values())}

    def _calculate_entity_confidence(self, text: str, entity_text: str,
                                  entity_type: str, base_score: float) -> float:
        """Calculate confidence score for each entity based on multiple factors."""
        confidence = base_score

        context_score = self._get_context_score(text, entity_text, entity_type)
        confidence = confidence * 0.7 + context_score * 0.3

        if len(entity_text.split()) > 1:
            confidence *= 1.1

        if entity_text.istitle():
            confidence *= 1.1

        if entity_type == "PERSON":
            if len(entity_text.split()) == 2:
                confidence *= 1.1
            if any(title in entity_text for title in ["Mr.", "Mrs.", "Ms.", "Dr."]):
                confidence *= 1.1
        elif entity_type == "WORK_OF_ART":
            if "movie" in text.lower() or "film" in text.lower():
                confidence *= 1.1
            if any(article in entity_text for article in ["The", "A", "An"]):
                confidence *= 1.05
        elif entity_type == "GENRE":
            common_genres = {"action", "comedy", "drama", "horror", "thriller",
                           "romance", "documentary", "animation", "sci-fi"}
            if entity_text.lower() in common_genres:
                confidence *= 1.2
        elif entity_type == "ORG":
            if any(suffix in entity_text for suffix in ["Studios", "Productions", "Entertainment"]):
                confidence *= 1.15
        elif entity_type == "ATTRIBUTE":
            common_attributes = {"excellent", "terrible", "amazing", "boring",
                               "fantastic", "disappointing"}
            if entity_text.lower() in common_attributes:
                confidence *= 1.1

        return min(max(confidence, 0.0), 1.0)

    def _get_context_score(self, text: str, entity_text: str, entity_type: str) -> float:
        """Analyze surrounding context to adjust confidence score."""
        context_score = 0.7

        lower_text = text.lower()
        context_keywords = {
            "PERSON": ["actor", "director", "stars", "plays", "directed by", "starring",
                      "performance by", "cast", "actress", "filmmaker"],
            "WORK_OF_ART": ["movie", "film", "show", "series", "watched", "sequel",
                           "trilogy", "adaptation", "premiere", "release"],
            "ORG": ["studio", "productions", "entertainment", "pictures", "company",
                   "distributor", "producer", "corporation"],
            "GENRE": ["genre", "type of", "kind of", "similar to", "category",
                     "style of", "like other"],
            "ATTRIBUTE": ["rating", "review", "quality", "style", "performance",
                         "visual", "story", "plot", "acting", "effects"]
        }

        keywords = context_keywords.get(entity_type, [])
        for keyword in keywords:
            if keyword in lower_text:
                context_score += 0.1

        entity_position = text.find(entity_text)
        if entity_position != -1:
            context_window = text[max(0, entity_position-30):
                                min(len(text), entity_position+30)]
            for keyword in keywords:
                if keyword in context_window.lower():
                    context_score += 0.15

        sentences = text.split('.')
        for sentence in sentences:
            if entity_text in sentence:
                if any(keyword in sentence.lower() for keyword in keywords):
                    context_score += 0.1

        return min(context_score, 1.0)

    def _analyze_message(self, message_id: int, conversation_id: str, text: str,
                        original_text: str, time_offset: int,
                        referenced_movies: List[str]) -> MessageAnalysisResult:
        """Analyze a single message for intent and entities."""
        try:
            entity_results = self._process_with_prompt_ner(text, self.entity_prompt_template.format(text=text))
            intent_results = self._process_with_prompt_intent(text, self.intent_prompt_template.format(text=text))

            return MessageAnalysisResult(
                message_id=message_id,
                conversation_id=conversation_id,
                text=original_text,
                time_offset=time_offset,
                intent=intent_results['intent'],
                confidence=intent_results['confidence'],
                entities=entity_results['entities'],
                referenced_movies=referenced_movies,
                analysis_details={'processed_text': text}
            )

        except Exception as e:
            self.logger.error(f"Error in message analysis: {str(e)}")
            return MessageAnalysisResult(
                message_id=message_id,
                conversation_id=conversation_id,
                text=original_text,
                time_offset=time_offset,
                intent="unknown",
                confidence=0.0,
                entities=[],
                referenced_movies=referenced_movies,
                analysis_details={'error': str(e)}
            )

    def process_jsonl_file(self, input_file: str, batch_size: int = 4) -> List[MessageAnalysisResult]:
        """Process entire JSONL file and analyze all messages."""
        all_results = []
        self.logger.info(f"Processing JSONL file: {input_file}")

        try:
            with jsonlines.open(input_file) as reader:
                dialogues = list(reader)

            for start in tqdm(range(0, len(dialogues), batch_size)):
                batch = dialogues[start:start + batch_size]
                for dialogue in batch:
                    dialogue_results = self._process_dialogue(dialogue)
                    all_results.extend(dialogue_results)

            self.logger.info(f"Processed {len(all_results)} messages from {len(dialogues)} dialogues")
            return all_results

        except Exception as e:
            self.logger.error(f"Error processing JSONL file: {str(e)}")
            raise

    def export_results_to_csv(self, results: List[MessageAnalysisResult], output_file: str) -> None:
        """Export analysis results to a CSV file with confidence scores."""
        self.logger.info(f"Exporting results to CSV file: {output_file}")

        try:
            data = [{
                "message_id": result.message_id,
                "conversation_id": result.conversation_id,
                "text": result.text,
                "time_offset": result.time_offset,
                "intent": result.intent,
                "intent_confidence": result.confidence,
                "entities": json.dumps(result.entities),
                "referenced_movies": json.dumps(result.referenced_movies),
                "processed_text": result.analysis_details.get('processed_text', ''),
                "error": result.analysis_details.get('error', '')
            } for result in results]

            df = pd.DataFrame(data)
            df.to_csv(output_file, index=False)

            self.logger.info(f"Exported {len(results)} results to {output_file}")
            self.logger.info(f"Average intent confidence: {df['intent_confidence'].mean():.3f}")

            entity_counts = df['entities'].apply(json.loads).apply(len)
            self.logger.info(f"Average entities per message: {entity_counts.mean():.2f}")

            self.logger.info("Results exported successfully")

        except Exception as e:
            self.logger.error(f"Error exporting results to CSV: {str(e)}")
            raise

    def get_statistics(self, results: List[MessageAnalysisResult]) -> Dict:
        """Generate statistics about the analysis results."""
        try:
            total_messages = len(results)
            total_entities = sum(len(result.entities) for result in results)

            intent_distribution = {}
            for result in results:
                intent_distribution[result.intent] = intent_distribution.get(result.intent, 0) + 1

            entity_type_distribution = {}
            entity_confidence_by_type = {}
            for result in results:
                for entity in result.entities:
                    entity_type = entity['type']
                    entity_type_distribution[entity_type] = entity_type_distribution.get(entity_type, 0) + 1

                    if entity_type not in entity_confidence_by_type:
                        entity_confidence_by_type[entity_type] = []
                    entity_confidence_by_type[entity_type].append(entity['confidence'])

            avg_confidence_by_type = {
                entity_type: sum(confidences) / len(confidences)
                for entity_type, confidences in entity_confidence_by_type.items()
            }

            return {
                "total_messages": total_messages,
                "total_entities": total_entities,
                "avg_entities_per_message": total_entities / total_messages if total_messages > 0 else 0,
                "intent_distribution": intent_distribution,
                "entity_type_distribution": entity_type_distribution,
                "avg_confidence_by_type": avg_confidence_by_type
            }

        except Exception as e:
            self.logger.error(f"Error generating statistics: {str(e)}")
            return {}

def main(input_file: str, output_file: str, batch_size: int = 4):
    """Main function to run the movie dialog analysis."""
    try:
        analyzer = MovieDialogAnalyzer()
        results = analyzer.process_jsonl_file(input_file, batch_size=batch_size)

        analyzer.export_results_to_csv(results, output_file)

        stats = analyzer.get_statistics(results)
        print("\nAnalysis Statistics:")
        print(f"Total messages processed: {stats['total_messages']}")
        print(f"Total entities found: {stats['total_entities']}")
        print(f"Average entities per message: {stats['avg_entities_per_message']:.2f}")

        print("\nIntent Distribution:")
        for intent, count in stats['intent_distribution'].items():
            print(f"{intent}: {count}")

        print("\nEntity Type Distribution:")
        for entity_type, count in stats['entity_type_distribution'].items():
            print(f"{entity_type}: {count} (Avg confidence: {stats['avg_confidence_by_type'][entity_type]:.3f})")

        print("\nProcessing and export completed successfully.")

    except Exception as e:
        print(f"Error in main execution: {str(e)}")
        raise

if __name__ == "__main__":
    input_file = 'train.jsonl'  # Input JSONL file path
    output_file = 'analysis_results.csv'  # Output CSV file path
    main(input_file, output_file)

2024-11-07 07:02:00,915 - __main__ - INFO - Setting up models...
2024-11-07 07:02:00,915 - __main__ - INFO - Setting up models...
2024-11-07 07:02:00,915 - __main__ - INFO - Setting up models...
2024-11-07 07:02:00,915 - __main__ - INFO - Setting up models...
2024-11-07 07:02:00,915 - __main__ - INFO - Setting up models...
2024-11-07 07:02:00,915 - __main__ - INFO - Setting up models...
2024-11-07 07:02:00,915 - __main__ - INFO - Setting up models...
INFO:__main__:Setting up models...
2024-11-07 07:02:04,580 - __main__ - INFO - Models loaded successfully
2024-11-07 07:02:04,580 - __main__ - INFO - Models loaded successfully
2024-11-07 07:02:04,580 - __main__ - INFO - Models loaded successfully
2024-11-07 07:02:04,580 - __main__ - INFO - Models loaded successfully
2024-11-07 07:02:04,580 - __main__ - INFO - Models loaded successfully
2024-11-07 07:02:04,580 - __main__ - INFO - Models loaded successfully
2024-11-07 07:02:04,580 - __main__ - INFO - Models loaded successfully
INFO:__main__


Analysis Statistics:
Total messages processed: 21
Total entities found: 71
Average entities per message: 3.38

Intent Distribution:
Ask for Recommendation: 5
Feedback on Recommendation: 2
Obtain Explanation: 3
Provide Preferences: 4
Accept Recommendation: 6
Chit-chat: 1

Entity Type Distribution:
ATTRIBUTE: 52 (Avg confidence: 0.940)
GENRE: 19 (Avg confidence: 0.857)

Processing and export completed successfully.
